<a href="https://colab.research.google.com/github/SantiagoVieiura/Lab2EDD2/blob/main/Lab2_edd2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def leerArchivo(nombre):
  df = pd.read_csv(nombre)
  return df

Creación del nodo


In [ ]:
from graphviz import Digraph
import csv

class Nodo:
    def __init__(self,code,SAname, SAcity, SAcountry, SAlatitude, SAlongitude ,DAcode,SAname, DAcity, DAcountry, DAlatitude, DAlongitude ):
        self.code = code
        self.SAname = SAname
        self.SAcity = SAcity
        self.SAcountry = SAcountry
        self.SAlatitude = SAlatitude
        self.SAlongitude = SAlongitude
        self.DAcode = DAcode
        self.DAname = DAname
        self.DACity = DACity
        self.DAcountry = DAcountry
        self.DAlatitude = DAlatitude
        self.DAlongitude = DAlongitude





Haversine formula :😡


In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])


    dlat = lat2 - lat1
    dlon = lon2 - lon1


    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))


    R = 6371
    distance = R * c

    return distance

codigo para saber si el grafo es conexo 😏

In [ ]:
def is_connected(graph):
    visited = set()

    def dfs(node):
        if node in visited:

            return visited.add(node)

        for neighbor in graph[node]:
            dfs(neighbor)

    # Start DFS from the first node
    start_node = next(iter(graph))
    dfs(start_node)

    return len(visited) == len(graph)

si entendi